<a href="https://colab.research.google.com/github/Skander28/Models/blob/main/Marbert(newdata).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.  

    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi
    
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4
Wed May 10 10:13:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                            

In [2]:
# install needed libraries
!pip install pyarabic
!pip install emoji
!pip install pystemmer
!pip install optuna==2.3.0
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 2.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234911 sha256=a2130ae8192fc01c637acf777667b7e6619ab94eff784c1ba36deb1b19acbe6f
  Stored in directory: /root/.cache/pip/wheels/02/3d/88/51a592b9ad17e7899126563698b4e3961983ebe85747228ba6
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 25.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pystemmer: filename=PyStemmer-2.2.0.1-cp310-cp310-linux_x86_64.whl size=58484

In [6]:
# import needed libraries
import numpy as np
import pandas as pd
import pyarabic.araby as ar

import re , emoji, Stemmer, functools, operator, string
import torch , optuna, gc, random, os

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample

import logging

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

In [7]:
from google.colab import drive
drive.mount('/content/drive')
     
import pandas as pd
tweets = pd.read_csv('/content/drive/MyDrive/NEWData.csv')
tweets.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,id,text,word_count,char_count,avg_char_per_word,stopwords,emoji_count,clean_text,dialect
0,0,1002651298255261568,@Sharehanart يمه الزين 😍 ربي يحفظك 😘😘,7.0,37.0,4.428571,0.0,3.0,يمه الزين ربي يحفظك,DZ
1,1,1031909335482417152,@azeezAzmm وأنت بألف خير عيني.. ينعاد عليكم با...,12.0,67.0,4.666667,1.0,2.0,وانت بالف خير عيني ينعاد عليكم بالصحه والعافيه...,DZ
2,2,1031910848443437056,@Mohmadridi6 امين يارب\nوصحا عيدك كل عام وانت ...,9.0,53.0,4.400000,1.0,3.0,امين يارب وصحا عيدك كل عام وانت بخير,DZ
3,3,1033650943387156352,@saraup01 بصحتك وربي يجيب شفاا وانتي علاه تكثر...,10.0,57.0,4.800000,0.0,5.0,بصحتك وربي يجيب شفاا وانتي علاه تكثري منو,DZ
4,4,1038485382818656384,ماعاد نجامل أحد خلي الناس تعرف انها خرا 💩💩,9.0,42.0,3.777778,1.0,2.0,ماعاد نجامل احد خلي الناس تعرف انها خرا,DZ


In [8]:
def data_cleaning (text):
  try:
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'^http?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"https\S+", "", text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub("(\s\d+)","",text) 
    text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "", text)
    text = re.sub("\d+", " ", text)
    text = text.replace("#", " ");
    text = text.replace("@", " ");
    text = text.replace("_", " ");
    translator = str.maketrans('', '', string.punctuation)
  except:
    return text
   
  return text

In [9]:
# Cleaning Training Data 
tweets['text'] = tweets['text'].apply(lambda x:   data_cleaning(x))

In [10]:
# a class representing the dataset
class Dataset:
    def __init__(
        self,
        name,
        train,
        test,
        label_list,
    ):
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list

In [11]:
class BERTModelDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTModelDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
  
    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())
    
      encoded_review = self.tokenizer.encode_plus(
      text,
      max_length= self.max_len,
      add_special_tokens= True,
      return_token_type_ids=False,
      pad_to_max_length=True,
      truncation='longest_first',
      return_attention_mask=True,
      return_tensors='pt'
    )
      input_ids = encoded_review['input_ids'].to(device)
      attention_mask = encoded_review['attention_mask'].to(device)

      return InputFeatures(input_ids=input_ids.flatten(), attention_mask=attention_mask.flatten(), label=self.label_map[self.target[item]])

In [13]:
Extra_Len = 6 # an extra padding in length , found to be useful for increasing F-score
Max_Len = tweets["text"].str.split().str.len().max() + Extra_Len

print(Max_Len)

#Spliting the Training data
Test_Size = 0.15
Rand_Seed = 42 

# Split original data into train and test sets
train_set, test_set = train_test_split(tweets, test_size=Test_Size, random_state=Rand_Seed + 1)

# Split training data into train and validation sets
train_set, evaluation_set = train_test_split(train_set, test_size=Test_Size, random_state=Rand_Seed)

print("Train set: ")
print(train_set["dialect"].value_counts())
print("---------------------------")
print ("Evaluation set: ")
print (evaluation_set["dialect"].value_counts())
print("---------------------------")
print ("test set: ")
print (test_set["dialect"].value_counts())

94
Train set: 
TN    11738
MA    11683
LY    11649
DZ    11629
Name: dialect, dtype: int64
---------------------------
Evaluation set: 
MA    2088
LY    2072
DZ    2042
TN    2039
Name: dialect, dtype: int64
---------------------------
test set: 
DZ    2488
LY    2438
MA    2388
TN    2382
Name: dialect, dtype: int64


In [14]:
Model_Used = "UBC-NLP/MARBERT"
Task_Name = "classification"

def model_init():
  return AutoModelForSequenceClassification.from_pretrained(Model_Used, return_dict=True, num_labels=len(label_map))

def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  print(classification_report(p.label_ids,preds))
  #print(confusion_matrix(p.label_ids,preds))

  macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[1,2])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {
      'macro_f1' : macro_f1,
      'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
  }

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [18]:
label_list = list(train_set["dialect"].unique())
#sentence_list = ['DZ', 'TN', 'MA', 'LY']
#ew_label_list = [sentence_list[label] for label in label_list]
print(label_list)
print(train_set["dialect"].value_counts())

data_set = Dataset( "OLY", train_set, evaluation_set, label_list )

# Define the list of class names
#label_list = ['DZ', 'TN', 'MA', 'LY']

# Create a label map that maps class names to their corresponding indices
label_map = {v: i for i, v in enumerate(label_list)}

# Print the label map
print(label_map)

train_dataset = BERTModelDataset(train_set["text"].to_list(),
                                 train_set["dialect"].to_list(),Model_Used,int(Max_Len),label_map)

evaluation_dataset = BERTModelDataset(evaluation_set["text"].to_list(),
                                      evaluation_set["dialect"].to_list(),Model_Used,int(Max_Len),label_map)

test_dataset = BERTModelDataset(test_set["text"].to_list(),
                                      test_set["dialect"].to_list(),Model_Used,int(Max_Len),label_map)

['LY', 'MA', 'TN', 'DZ']
TN    11738
MA    11683
LY    11649
DZ    11629
Name: dialect, dtype: int64
{'LY': 0, 'MA': 1, 'TN': 2, 'DZ': 3}


In [19]:
#define training arguments
training_args = TrainingArguments("./train")
training_args.lr_scheduler_type = 'cosine'
training_args.evaluate_during_training = True
training_args.adam_epsilon =1e-8 
training_args.learning_rate = 2e-05
training_args.fp16 = True
training_args.per_device_train_batch_size = 64
training_args.per_device_eval_batch_size = 32
training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 4
training_args.warmup_steps = 0 
training_args.evaluation_strategy = EvaluationStrategy.EPOCH
training_args.seed = 42 
training_args.disable_tqdm = False

In [20]:
training_args.dataloader_pin_memory = False
gc.collect()
torch.cuda.empty_cache()
set_seed(Rand_Seed) 

trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset= evaluation_dataset,
    compute_metrics=compute_metrics
)



print(training_args.seed)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at U

42


In [21]:

import os
os.environ["WANDB_DISABLED"] = "true"
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch,Training Loss,Validation Loss,Macro F1,Macro F1 Pos Neg,Macro Precision,Macro Recall,Accuracy
1,No log,0.191251,0.935862,0.976888,0.937208,0.935260,0.935445
2,0.234800,0.184794,0.941244,0.978899,0.942521,0.940943,0.941148
3,0.090800,0.213003,0.942808,0.975991,0.942952,0.942721,0.942847
4,0.090800,0.238344,0.942211,0.975999,0.942351,0.942124,0.942240


              precision    recall  f1-score   support

           0       0.88      0.94      0.91      2072
           1       1.00      0.97      0.98      2088
           2       0.99      0.95      0.97      2039
           3       0.89      0.88      0.88      2042

    accuracy                           0.94      8241
   macro avg       0.94      0.94      0.94      8241
weighted avg       0.94      0.94      0.94      8241



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

              precision    recall  f1-score   support

           0       0.88      0.95      0.91      2072
           1       1.00      0.97      0.99      2088
           2       0.98      0.97      0.97      2039
           3       0.91      0.88      0.89      2042

    accuracy                           0.94      8241
   macro avg       0.94      0.94      0.94      8241
weighted avg       0.94      0.94      0.94      8241



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

              precision    recall  f1-score   support

           0       0.91      0.93      0.92      2072
           1       0.99      0.98      0.98      2088
           2       0.97      0.97      0.97      2039
           3       0.90      0.90      0.90      2042

    accuracy                           0.94      8241
   macro avg       0.94      0.94      0.94      8241
weighted avg       0.94      0.94      0.94      8241



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


              precision    recall  f1-score   support

           0       0.91      0.92      0.92      2072
           1       0.99      0.98      0.98      2088
           2       0.97      0.97      0.97      2039
           3       0.90      0.90      0.90      2042

    accuracy                           0.94      8241
   macro avg       0.94      0.94      0.94      8241
weighted avg       0.94      0.94      0.94      8241



TrainOutput(global_step=1460, training_loss=0.12420918190315978, metrics={'train_runtime': 949.4421, 'train_samples_per_second': 196.743, 'train_steps_per_second': 1.538, 'total_flos': 9023444676422592.0, 'train_loss': 0.12420918190315978, 'epoch': 4.0})

In [22]:
# Evaluate test set
test_results = trainer.predict(test_dataset)

# Print test set results
print(test_results.metrics)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


              precision    recall  f1-score   support

           0       0.91      0.92      0.92      2438
           1       0.99      0.98      0.99      2388
           2       0.97      0.96      0.97      2382
           3       0.91      0.91      0.91      2488

    accuracy                           0.94      9696
   macro avg       0.94      0.94      0.94      9696
weighted avg       0.94      0.94      0.94      9696

{'test_loss': 0.23749323189258575, 'test_macro_f1': 0.944371009814825, 'test_macro_f1_pos_neg': 0.9759032322498589, 'test_macro_precision': 0.9446503610591757, 'test_macro_recall': 0.9441439047732056, 'test_accuracy': 0.9436881188118812, 'test_runtime': 16.5449, 'test_samples_per_second': 586.042, 'test_steps_per_second': 18.314}


In [23]:
# Save the trained model to a file
torch.save(trainer.model.state_dict(), "trainer.pth")

In [27]:
import torch
from transformers import AutoTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(Model_Used, num_labels=4).to(device)

# Load the trained model from disk
model_state_dict = torch.load('/content/trainer.pth')
model.load_state_dict(model_state_dict)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(Model_Used)



# Tokenize the input sentence
input_sentence = "نبي نروح للحوش "
#input_sentence = 'واش دير للعشى'
#input_sentence  = "شوكران علا هاد "
#input_sentence = "فرحان و نتشرف الي انا من اقلية الشعب الي عمري ما صدقت كذبة الثورة و الديمقراطية و ما عداها عليا حتى سياسي ملي حكمو من 2011 و ما تحكموش في تفكيري باجات فيسبوك و اعلاميين مرتزقة و سبر اراء غالط"
tokenized_input = tokenizer(input_sentence, padding=True, truncation=True, max_length=Max_Len, return_tensors='pt')
input_ids = tokenized_input['input_ids'].to(device)
attention_mask = tokenized_input['attention_mask'].to(device)

# Pass the input tensor through the model to obtain the predictions
outputs = model(input_ids=input_ids, attention_mask=attention_mask)
predicted_labels = torch.argmax(outputs.logits, dim=1)
confidence_score = torch.softmax(outputs.logits, dim=1)[0][predicted_labels]

# Print the predicted class and its corresponding label
#{'LY': 0, 'MA': 1, 'TN': 2, 'DZ': 3}
label_map = {0: "LY", 1: "MA", 2: "TN", 3: "DZ"}
predicted_class = label_map[predicted_labels.item()]
print("Predicted class:", predicted_class)
print("Confidence score:", confidence_score.item())


Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at U

Predicted class: LY
Confidence score: 0.997211754322052
